In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yfinance



SyntaxError: invalid syntax (2977875142.py, line 1)

In [ ]:
import yfinance as yf
import pandas as pd

def download_and_resample(symbol, start_date, end_date, interval):
    try:
        print(f"\n📥 Downloading {symbol} data from {start_date} to {end_date} ({interval})...")

        # Download OHLCV data
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)

        if df.empty:
            print("❌ No data found. Check your symbol, date range, or interval.")
            return

        # Reset index to separate date & time
        df.reset_index(inplace=True)

        # Detect correct datetime column name
        datetime_col = 'Datetime' if 'Datetime' in df.columns else 'Date'
        df['date'] = pd.to_datetime(df[datetime_col]).dt.date
        df['time'] = pd.to_datetime(df[datetime_col]).dt.time

        # Select and rename columns
        df = df[['date', 'time', 'Open', 'High', 'Low', 'Close', 'Volume']]
        df.columns = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']

        # Drop missing values
        df.dropna(inplace=True)

        # Combine date and time for resampling
        df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
        df.set_index('datetime', inplace=True)

        # ✅ Map Yahoo interval → Pandas resample frequency
        interval_map = {
            '1m': '1T', '2m': '2T', '5m': '5T', '15m': '15T', '30m': '30T',
            '60m': '60T', '90m': '90T', '1h': '1H',
            '1d': '1D', '5d': '5D', '1wk': '1W', '1mo': '1M', '3mo': '3M'
        }

        pandas_interval = interval_map.get(interval, '1D')

        # Resample
        resampled = df.resample(pandas_interval).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()

        # Extract date/time
        resampled['date'] = resampled.index.date
        resampled['time'] = resampled.index.time
        resampled = resampled[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]

        # Save to CSV
        filename = f"{symbol}_{interval}.csv"
        resampled.to_csv(filename, index=False)

        print(f"✅ File saved: {filename}")
        print("\n📊 Sample Data:\n", resampled.head())

    except Exception as e:
        print(f"⚠️ Error: {e}")


# ========== MAIN PROGRAM ==========
if __name__ == "__main__":
    symbol = input("Enter Stock Symbol (e.g., SBIN.NS, RELIANCE.NS): ").strip()
    start_date = input("Enter Start Date (YYYY-MM-DD): ").strip()
    end_date = input("Enter End Date (YYYY-MM-DD): ").strip()
    interval = input("Enter Timeframe (e.g., 5m, 15m, 1h, 1d): ").strip()

    valid_intervals = ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']
    if interval not in valid_intervals:
        print("❌ Invalid interval! Please choose one of:", valid_intervals)
    else:
        download_and_resample(symbol, start_date, end_date, interval)

        